In [5]:
import os
import sys
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
#from trl import SFTTrainer
#from peft import LoraConfig, PeftModel
#import trl
from trl import SFTTrainer
import peft
from peft import LoraConfig
import accelerate 
import bitsandbytes
import transformers

print("Accelerate version:", accelerate.__version__)
print("Peft version:", peft.__version__)
#print("BitsAndBytes version:", bitsandbytes.__version__)
#print("Transformers version:", transformersformers.__version__)

Accelerate version: 0.21.0
Peft version: 0.4.0


In [6]:
#
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
# Further package issues:: tried various things. some worked, some not -- will find workaround.

#bin_path = "/home/ssharp/.local/bin"
#os.environ['PATH'] = f"{bin_path}:{os.environ['PATH']}"
#sys.path.append(bin_path)

In [7]:
# The model to train, based on HuggingFace naming-conventions
# Originally selected model incompliant - no available config file to feed the transformers wrapper
#model_name = "aptha/Meta-Llama-3-8B-Instruct-Q4_0-GGUF"
model_name = "NousResearch/llama-2-7b-chat-hf"

# The instruction dataset to use:
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
#new_model = "llama-3-Llamaguaco"
new_model = "llama-2-mamma"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.2

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = True

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
#output_dir = "./Llama_3_results"
output_dir = "model_results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 5

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 100

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
#max_seq_length = None
max_seq_length = 8192

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [8]:
# Load dataset
dataset = load_dataset(dataset_name, cache_dir=None, split="train")

# Activate 4-bit precision base model loading:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = True

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=use_4bit,
#    bnb_4bit_quant_type=bnb_4bit_quant_type,
#    bnb_4bit_compute_dtype=compute_dtype,
#    bnb_4bit_use_double_quant=use_nested_quant,
#)


# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8: 
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
#model = AutoModelForCausalLM.from_pretrained(
#    model_name,
#    quantization_config=bnb_config,
#    device_map=device_map
#)


max_memory = {i: '12000MB' for i in range(torch.cuda.device_count())}

quant_config = BitsAndBytesConfig(
        load_in_4bit = use_4bit,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )

# AutoModel selects most appropriate model configuration for -model_name-
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #"aptha/Meta-Llama-3-8B-Instruct-Q4_0-GGUF/blob/main/Meta-Llama-3-8B-Instruct-Q4_0.gguf",
    cache_dir=None,
    #load_in_4bit=True,
    device_map='auto',
    max_memory=max_memory,
    #torch_dtype=torch.bfloat16,
    quantization_config=quant_config
)

# Load LLaMA tokenizer for use with AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=None, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


model.config.use_cache = False
model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/home/ssharp/.local/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Saving Trained Model:

In [10]:

############ Train & Save Model  ####################

#trainer.train()
#trainer.model.save_pretrained(new_model)

#### Comment out to prevent accidentally over-writing saved model. ####

# This is a checkpoint.

### Post-training should be defined below: Load a model from local memory then run inference.

In [10]:
#test_model = trainer.model.load_pretrained

#model_path = "/"

# Load the model

#config = transformers.AutoConfig.from_pretrained("./llama-2-mamma/")
#loaded_model = AutoModelForCausalLM.from_pretrained("./llama-2-mamma/")


# Loading Quantized Model is different per each wrapper distortion --
# using peft: load Quantized FineTune_Model = Base_model + Tuned Adapter : still need to load base model seperately

#from peft import PeftModel, PeftConfig
import torch
import peft
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, init_empty_weights

#model_name = "NousResearch/llama-2-7b-chat-hf" # Base Model to test
#base_model = AutoModelForCausalLM.from_pretrained(model_name)
peft_model_name = "./llama-2-mamma/"

#saved info on fine-tune includes reference to base_model name
config = peft.PeftConfig.from_pretrained(peft_model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)

# Tokenizer is not saved automatically : will just re-define the previous definition since it doesnt change.
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, cache_dir=None, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix for an overflow issue with the fp16 training


# Load the Quantized Lora model from the saved parameters (post-training)
test_model = peft.PeftModel.from_pretrained(base_model, peft_model_name, is_trainable=False) #Flag: Inference
test_model.print_trainable_parameters()

# Here we can see that this trained model uses ~6.8B Params for base model + 33.6M Params for the adapter.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 0 || all params: 6,771,970,048 || trainable%: 0.0


In [3]:
test_model = peft.PeftModel.from_pretrained(base_model, peft_model_name, is_trainable=False) #Flag: Inference

In [11]:
##### Try to sample model outputs: #####

print(type(test_model))
# Sample messages to forward:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who responds in pirate speak or in sea-shanties."},
    {"role": "user", "content": "What is your quest?"},
]

sample_text = "Can you sing me a sea shanty?"

# Causal models:
#tokenized_prompt = tokenizer(prompt, return_tensors="pt").to("cuda")
#outputs = model.generate(**tokenized_prompt)
# PEFT models:
input_ids= tokenizer(sample_text, return_tensors="pt")#.input_ids.to("cuda")
outputs = test_model.generate(input_ids=input_ids)
tokenizer.decode(outputs)



# Tokenize input text
#input_ids = tokenizer.encode(messages, return_tensors="pt")
#input_ids = test_model.tokenizer.encode(sample_text, return_tensors="pt") 


# Generate output tokens
#with torch.no_grad():
    #output = test_model.generate(input_ids, max_length=8192, num_return_sequences=1)
    #output = test_model.generate(input_ids, max_length=8192)

# Decode output tokens into text
#output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print output text
print(output_text)


# # Formatting to send messages through tokenizer
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(test_model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)
# # https://github.com/Lightning-AI/litgpt/issues/327

# # Frame output formatting:
# outputs = test_model.generate(
#     input_ids,
#     max_new_tokens=128,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )

# # Push Sample Message through the model, display output
# response = outputs[0][input_ids.shape[-1]:]
# print(tokenizer.decode(response, skip_special_tokens=True))

<class 'peft.peft_model.PeftModelForCausalLM'>


/home/ssharp/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


AttributeError: 

In [12]:
# Try to measure memory used:
#from transformers import GPT2LMHeadModel
import psutil
import torch
#from torchsummary import summary

# Need to find what "input_size" references here for an LLM
#summary(test_model, input_size=(input_shape))


# Get memory usage (bytes) 
memory_usage = psutil.Process().memory_info().rss
print("Memory Usage:", memory_usage, "bytes")
# This is not useful, just returns available RAM being used by the alloted datahub kernel.

# device isn't a hard/specific handler here.
# (in Linux Terminal) nvidia-smi :doesn't refer consumed GPU since its not actively running while waiting.
# need to devize a way to measure a GPU peak* consumption?

#test_model.to(device="cuda")
#print("torch.cuda summary: ",torch.cuda.memory_summary(device=device_map))
memory_allocated = torch.cuda.memory_allocated(device="cuda")
print("CUDA Memory allocated (bytes):", memory_allocated)

Memory Usage: 13205839872 bytes
CUDA Memory allocated (bytes): 9032909824


#### Note: 
- Many wrappers are leading ambiguity -- unsure where only one wrapper around functional model, where many wrappers, or where wrappers are modified copies of other wrappers.

- when loading model from parameters, the Datahub Kernel keeps dying (not consistently, but commonly) -- may be some form of memory issue or numerical instability. 

- our new, Quantized fine-tune model must be loaded alongside the original model. This does seem efficient to save space for those that try many fine-tunes for the same models. _This may be good to use to exploit larger models with special objectives_: could this be a common form MoE models use?

- Model Params: [trainable params: 33,554,432 || all params: 6,771,970,048 || trainable%: 0.49548996469513035] This tells us that the model is uring 6.8B Parameters for the base model + 33.6M params for the adapter. This is useful information for our starting place.

# Trial 2:
To try to force a Llama-3-8B model into a Q4-quant for compliance to our memory constraints:

In [28]:
# ### Repository Issues, but this should provide a good sample Format:
# #Initial guide found in source: https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04
# !pip install huggingface_hub

# from huggingface_hub import notebook_login
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )

# # Sample messages to forward:
# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak or Sea-shanties!"},
#     {"role": "user", "content": "Who are you?"},
# ]

# # Formatting to send messages through tokenizer
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)
# # https://github.com/Lightning-AI/litgpt/issues/327

# # Frame output formatting:
# outputs = model.generate(
#     input_ids,
#     max_new_tokens=128,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )

# # Push Sample Message through the model, display output
# response = outputs[0][input_ids.shape[-1]:]
# print(tokenizer.decode(response, skip_special_tokens=True))

#Doesnt run due to more repository compliance | compatibility issues. 